In [1]:
import pandas as pd
import json
import random
import uuid
from datetime import datetime, timedelta

def load_base_companies(file_path):
    """
    Attempts to load real company names from a Kaggle CSV.
    If the file doesn't exist, it generates placeholders.
    """
    try:
        df = pd.read_csv('kaggle_companies.csv')
        # Assuming Kaggle column is 'name' or 'Company Name'
        # We take the first 50 unique names
        # Change 'name' to whatever the column is called in your Kaggle CSV
        names = df['Company Name'].dropna().unique()[:50].tolist()
        print(f"--- Loaded {len(names)} real company names from Kaggle base ---")
        return names
    except FileNotFoundError:
        print("--- Kaggle file not found. Falling back to synthetic names. ---")
        return [f"Kaggle_Fallback_{i}" for i in range(50)]

def generate_hybrid_data():
    # 1. Load the "Real" Base
    company_names = load_base_companies('kaggle_companies.csv')
    
    # 2. Setup Companies Metadata
    companies_meta = []
    tiers = ["Basic", "Pro", "Enterprise"]
    
    for name in company_names:
        comp_id = str(uuid.uuid4())[:8]
        companies_meta.append({
            "company_id": comp_id,
            "company_name": name,
            "subscription_tier": random.choice(tiers),
            "monthly_fee": random.randint(5000, 75000),
            "is_at_risk": random.random() < 0.2
        })

    # 3. Generate 5,000 Synthetic Logs
    logs = []
    features = ["AI_Resume_Screen", "Bulk_Email", "Dashboard_View", "Job_Post_Creation"]
    end_date = datetime.now()
    
    for _ in range(5000):
        comp = random.choice(companies_meta)
        
        # Churn Logic: 'At risk' companies have logs mostly older than 7 days
        if comp["is_at_risk"]:
            days_ago = random.uniform(7, 30) 
        else:
            days_ago = random.uniform(0, 30)
            
        timestamp = end_date - timedelta(days=days_ago)
        
        logs.append({
            "log_id": str(uuid.uuid4()),
            "user_id": f"user_{random.randint(100, 999)}",
            "company_id": comp["company_id"],
            "timestamp": timestamp.isoformat(),
            "feature_used": random.choice(features),
            "session_duration": random.randint(5, 45)
        })

    # 4. Save Outputs
    # Save Logs as JSON (Raw)
    with open('users_activity.json', 'w') as f:
        json.dump(logs, f, indent=4)
    
    # Save Metadata as CSV
    meta_df = pd.DataFrame(companies_meta).drop(columns=['is_at_risk'])
    meta_df.to_csv('account_meta.csv', index=False)
    
    print("Files 'users_activity.json' and 'account_meta.csv' are ready!")

if __name__ == "__main__":
    generate_hybrid_data()

--- Loaded 50 real company names from Kaggle base ---
Files 'users_activity.json' and 'account_meta.csv' are ready!


In [5]:
import pandas as pd
import json
import urllib.parse  # Required to handle the '@' in your password
from sqlalchemy import create_engine

# --- CONFIGURATION ---
DB_USER = "root"
DB_PASSWORD = "aadithian@2003" 
DB_HOST = "localhost"             # Changed from 'Data' to 'localhost'
DB_PORT = "3306"
DB_NAME = "saas_monitoring"

def run_etl_mysql():
    # 1. Load Data
    print("Reading raw data...")
    try:
        with open('users_activity.json', 'r') as f:
            logs_data = json.load(f)
        
        df_logs = pd.DataFrame(logs_data)
        df_meta = pd.read_csv('account_meta.csv')
    except FileNotFoundError as e:
        print(f"Error: Could not find data files. {e}")
        return

    # 2. Convert timestamps to datetime objects
    df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'])

    # 3. Feature Engineering: Define "AI Features"
    ai_features = ['AI_Resume_Screen']
    
    # Calculate Total Sessions and AI usage per company
    stats = df_logs.groupby('company_id').agg(
        total_sessions=('log_id', 'count'),
        ai_usage=('feature_used', lambda x: x.isin(ai_features).sum())
    ).reset_index()

    # 4. Calculate Health Score (0-100 scale)
    max_sessions = stats['total_sessions'].max() or 1
    max_ai = stats['ai_usage'].max() or 1

    stats['health_score'] = (
        (stats['total_sessions'] / max_sessions * 40) + 
        (stats['ai_usage'] / max_ai * 60)
    ).round(2)

    # 5. Merge with Metadata
    final_df = pd.merge(df_meta, stats, on='company_id', how='left')
    final_df['health_score'] = final_df['health_score'].fillna(0)

    # 6. Export to MySQL
    try:
        print(f"Connecting to MySQL database: {DB_NAME}...")
        
        # FIX: Percent-encode the password so the '@' doesn't break the connection string
        safe_password = urllib.parse.quote_plus(DB_PASSWORD)
        
        # Build the connection URL using the safe password and localhost
        connection_url = f"mysql+mysqlconnector://{DB_USER}:{safe_password}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
        engine = create_engine(connection_url)
        
        # Save the main Health Table
        print("Uploading 'account_health' table...")
        final_df.to_sql('account_health', engine, if_exists='replace', index=False)
        
        # Save the Raw Logs
        print("Uploading 'raw_logs' table...")
        df_logs.to_sql('raw_logs', engine, if_exists='replace', index=False)
        
        print("\nETL Complete! Open MySQL Workbench to query your data.")
        
    except Exception as e:
        print(f"An error occurred while connecting to MySQL: {e}")

if __name__ == "__main__":
    run_etl_mysql()

Reading raw data...
Connecting to MySQL database: saas_monitoring...
Uploading 'account_health' table...
Uploading 'raw_logs' table...

ETL Complete! Open MySQL Workbench to query your data.
